<a href="https://colab.research.google.com/github/asantos20/conciliacao-bancaria/blob/main/Detec%C3%A7%C3%A3o_fraude_concili%C3%A7%C3%A3o_banc%C3%A1ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalando a biblioteca

pip install fuzzywuzzy

In [3]:
# Biblioteca

import pandas as pd
from fuzzywuzzy import fuzz

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
# Simulação dos dados extraídos dos PDFs
contas_pagar = pd.DataFrame({
    'Fornecedor': ['ALFA SERVIÇOS', 'BETA TRANSPORTES', 'GAMA COMERCIAL'],
    'Valor': [1200.00, 980.00, 3000.00],
    'Data': ['2025-08-01', '2025-08-03', '2025-08-04']
})

extrato_bancario = pd.DataFrame({
    'Descrição': ['ALFA SERVIÇOS', 'BETA TRANSPORTES', 'DELTA SUPRIMENTOS'],
    'Valor': [1200.00, 980.00, 600.00],
    'Data': ['2025-08-01', '2025-08-03', '2025-08-05']
})

In [5]:
# Conciliação
resultado = []

for idx, row in contas_pagar.iterrows():
    match = extrato_bancario[
        (extrato_bancario['Valor'] == row['Valor']) &
        (extrato_bancario['Data'] == row['Data']) &
        (extrato_bancario['Descrição'].apply(lambda x: fuzz.partial_ratio(x, row['Fornecedor']) > 90))
    ]

    if not match.empty:
        resultado.append({
            'Fornecedor Previsto': row['Fornecedor'],
            'Valor Previsto': row['Valor'],
            'Data Prevista': row['Data'],
            'Fornecedor Extrato': match.iloc[0]['Descrição'],
            'Valor Realizado': match.iloc[0]['Valor'],
            'Data Realizada': match.iloc[0]['Data'],
            'Status': '✔️ Conciliado'
        })
    else:
        resultado.append({
            'Fornecedor Previsto': row['Fornecedor'],
            'Valor Previsto': row['Valor'],
            'Data Prevista': row['Data'],
            'Fornecedor Extrato': None,
            'Valor Realizado': None,
            'Data Realizada': None,
            'Status': '❌ Não pago'
        })

In [6]:
# Verificar pagamentos não previstos
for idx, row in extrato_bancario.iterrows():
    match = contas_pagar[
        (contas_pagar['Valor'] == row['Valor']) &
        (contas_pagar['Data'] == row['Data']) &
        (contas_pagar['Fornecedor'].apply(lambda x: fuzz.partial_ratio(x, row['Descrição']) > 90))
    ]

    if match.empty:
        resultado.append({
            'Fornecedor Previsto': None,
            'Valor Previsto': None,
            'Data Prevista': None,
            'Fornecedor Extrato': row['Descrição'],
            'Valor Realizado': row['Valor'],
            'Data Realizada': row['Data'],
            'Status': '⚠️ Não previsto'
        })

In [7]:
df_resultado = pd.DataFrame(resultado)
print(df_resultado)

  Fornecedor Previsto  Valor Previsto Data Prevista Fornecedor Extrato  \
0       ALFA SERVIÇOS          1200.0    2025-08-01      ALFA SERVIÇOS   
1    BETA TRANSPORTES           980.0    2025-08-03   BETA TRANSPORTES   
2      GAMA COMERCIAL          3000.0    2025-08-04               None   
3                None             NaN          None  DELTA SUPRIMENTOS   

   Valor Realizado Data Realizada           Status  
0           1200.0     2025-08-01    ✔️ Conciliado  
1            980.0     2025-08-03    ✔️ Conciliado  
2              NaN           None       ❌ Não pago  
3            600.0     2025-08-05  ⚠️ Não previsto  
